In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

In [6]:
# program reads in turnstile date from the mta site
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
# these are the lists containing the dates that the get_data function will read    
list2018 = [180324, 180331, 180407, 180414, 180421, 180428, 180505, 180512, 180519, 180526, 180602, 180609, 180616, 180623]
list2017 = [170325, 170401, 170408, 170415, 170422, 170429, 170506, 170513, 170520, 170527, 170603, 170610, 170617, 170624]
list2016 = [160326, 160402, 160409, 160416, 160423, 160429, 160507, 160514, 160521, 160528, 160604, 160611, 160618, 160625]

# read in data for turnstiles (select which year) and stations (downloaded csv locally) and make columns neat for turnstile data
turnstilesDF = get_data(list2018)
stationsDF = pd.read_csv('Stations.csv')
turnstilesDF.columns = [column.strip() for column in turnstilesDF.columns]

In [26]:
# in case trying to merge data sets by station, change "Stop Name" column in stationsDF to match"STATIONS"
# in turnstilesDF
stationsDF.rename(columns = {'Stop Name':'STATION'}, inplace=True)

# add new column which contains stations in edited format (all caps and no spaces) in order to create consistency
# with an "edited stations" column to merge with stationsDF over

# list of stations in Manhattan and put in edited format (all caps and no spaces) in order to create consistency
# with an "edited stations" column to be placed in DF with turnstile info which can then be merged on
manhattanStationsDF = stationsDF.loc[stationsDF.Borough == 'M']
uniqueStationsEdit = [station.upper() for station in manhattanStationsDF.STATION.unique()]
uniqueStationsEdit = [station.replace(' ','') for station in uniqueStationsEdit]

In [27]:
# create DATE_TIME column to check for duplicate entries
turnstilesDF['DATE_TIME'] = pd.to_datetime(turnstilesDF.DATE + ' ' + turnstilesDF.TIME, format = "%m/%d/%Y %H:%M:%S")
turnstilesDF.DATE_TIME

# check for duplicate entries in turnstilesDF
(turnstilesDF
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head()


# get daily entry totals
turnstilesEntriesDF = turnstilesDF.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).ENTRIES.first().reset_index()

turnstilesEntriesDF[["PREV_DATE", "PREV_ENTRIES"]] = (turnstilesEntriesDF
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .transform(lambda grp: grp.shift(1)))

turnstilesEntriesDF.dropna(subset=["PREV_DATE"], axis=0, inplace=True)


# get daily exit totals
turnstilesExitsDF = turnstilesDF.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).EXITS.first().reset_index()


turnstilesExitsDF[["PREV_DATE", "PREV_EXITS"]] = (turnstilesExitsDF
                                                   .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "EXITS"]
                                                   .transform(lambda grp: grp.shift(1)))

turnstilesExitsDF.dropna(subset=["PREV_DATE"], axis=0, inplace=True)


#combine entries and exits to get total (labeled as "traffic")
EntriesAndExitsDF = pd.merge(turnstilesEntriesDF, turnstilesExitsDF, on=["C/A", "UNIT", "SCP", "STATION", "DATE"])
EntriesAndExitsDF['TRAFFIC'] = EntriesAndExitsDF['PREV_ENTRIES'] + EntriesAndExitsDF['PREV_EXITS']


# get daily traffic totals into new DF called EntriesAndExitsDF
EntriesAndExitsDF = EntriesAndExitsDF.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.TRAFFIC.first().reset_index()

EntriesAndExitsDF["PREV_TRAFFIC"] = (EntriesAndExitsDF
                                      .groupby(["C/A", "UNIT", "SCP", "STATION"])["TRAFFIC"]
                                      .transform(lambda grp: grp.shift(1)))

EntriesAndExitsDF.dropna(subset=["PREV_TRAFFIC"], axis=0, inplace=True)

In [28]:
#function to determine daily traffic
def get_daily_counts(row, max_counter):
    counter = row["TRAFFIC"] - row["PREV_TRAFFIC"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # print(row["TRAFFIC"], row["PREV_TRAFFIC"]) --> if you want to see the specific case
        counter = min(row["TRAFFIC"], row["PREV_TRAFFIC"])
        # if current entries is bad, use yesterday's count as proxy
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

# If counter is > 1HundredThousand, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits


EntriesAndExitsDF["DAILY_TRAFFIC"] = EntriesAndExitsDF.apply(get_daily_counts, axis=1, max_counter=100000)

In [29]:
# add "edited station" column to EntriesAndExitsDF in order to filter for Manhattan stations
trafficStationsEdit = [station.upper() for station in EntriesAndExitsDF.STATION.unique()]
trafficStationsEdit = [station.replace(' ','') for station in trafficStationsEdit]
trafficStationsDF = pd.DataFrame({'STATION':trafficStations, 'EDITED_STATION':trafficStationsEdit})
EntriesAndExitsDF = pd.merge(EntriesAndExitsDF, trafficStationsDF, on='STATION')

# function that creates column that states if station is in Manhattan
def inManhattan(row):
    if row['EDITED_STATION'] in uniqueStationsEdit:
        return 'Y'
    else:
        return 'N'


#create new daily traffic DF with only Manhattan stations
EntriesAndExitsDF['MANHATTAN'] = EntriesAndExitsDF.apply(inManhattan, axis=1)
TrafficManhattanDF = EntriesAndExitsDF.loc[EntriesAndExitsDF.MANHATTAN == 'Y']

# some stations do not survive the Manhattan filter due to difference in string nomenclature (even with the
# made for "edited stations"), so 2 big ones are given their own DFs and concatenated into the filtered DF
grandCentralDF = newTrafficStationsDF.loc[EntriesAndExitsDF.STATION == 'GRD CNTRL-42 ST']
pennDF = newTrafficStationsDF.loc[EntriesAndExitsDF.STATION == '34 ST-PENN STA']

In [30]:
# final DF
TrafficManhattanDF = pd.concat([TrafficManhattanDF, grandCentralDF, pennDF])
TrafficManhattanDF.groupby(['C/A', 'UNIT', 'STATION', 'DATE']).sum().sort_values(['DAILY_TRAFFIC'], ascending=False)


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


DAILY_TRAFFIC  PREV_TRAFFIC  \
C/A   UNIT STATION         DATE                                      
R238  R046 GRD CNTRL-42 ST 06/16/2018       116596.0   280511788.0   
                           06/02/2018       114699.0   279150921.0   
                           06/15/2018       114698.0   280397090.0   
                           03/31/2018       114178.0   274247658.0   
                           06/22/2018       114065.0   281081956.0   
                           06/08/2018       113889.0   279716405.0   
                           03/30/2018       113343.0   274134315.0   
                           06/14/2018       113195.0   280283895.0   
                           06/09/2018       113185.0   279830294.0   
                           03/29/2018       113121.0   274021194.0   
                           06/21/2018       112395.0   280969561.0   
                           06/01/2018       112196.0   279038725.0   
                           06/07/2018       111518.0   279604887.0   
                           05/25/2018       111395.0   278441439.0   
                           05/18/2018       111209.0   278583843.0   
                           06/10/2018       111178.0   279943479.0   
                           04/07/2018       111030.0   274901980.0   
                           06/17/2018       110363.0   280628384.0   
                           04/21/2018       110280.0   276158796.0   
                           05/26/2018       110112.0   278552834.0   
                           05/12/2018       110065.0   278057096.0   
                           04/05/2018       109954.0   274682714.0   
                           05/24/2018       109724.0   278331715.0   
                           06/03/2018       109408.0   279265620.0   
                           04/20/2018       109355.0   276049441.0   
                           04/06/2018       109312.0   274792668.0   
                           04/28/2018       109032.0   276792603.0   
                           04/22/2018       108870.0   276269076.0   
                           05/20/2018       108612.0   278011761.0   
                           05/05/2018       108532.0   277416605.0   
...                                              ...           ...   
R242A R049 51 ST           03/26/2018            0.0     9453688.0   
                           03/27/2018            0.0     9453688.0   
                           04/02/2018            0.0     9480904.0   
R137  R031 34 ST-PENN STA  05/22/2018            0.0    16830181.0   
R113  R028 FULTON ST       05/29/2018            0.0   572684432.0   
R112  R027 WALL ST         06/04/2018            0.0    30730758.0   
R112A R027 WALL ST         04/24/2018            0.0    41084088.0   
R112  R027 WALL ST         06/05/2018            0.0    30730758.0   
                           06/11/2018            0.0    30802678.0   
                           06/12/2018            0.0    30802678.0   
                           06/18/2018            0.0    30874054.0   
                           06/19/2018            0.0    30874054.0   
R112A R027 WALL ST         03/20/2018            0.0    40895890.0   
                           03/27/2018            0.0    40930830.0   
                           04/03/2018            0.0    40969187.0   
                           04/10/2018            0.0    41005399.0   
                           04/17/2018            0.0    41044805.0   
                           05/01/2018            0.0    41122522.0   
R137  R031 34 ST-PENN STA  05/29/2018            0.0    16870104.0   
A082  R028 FULTON ST       03/20/2018            0.0    54918799.0   
R113  R028 FULTON ST       04/09/2018            0.0   572178485.0   
                           04/10/2018            0.0   572178485.0   
R137  R031 34 ST-PENN STA  06/19/2018            0.0    16985328.0   
                           06/18/2018            0.0    16985328.0   
                           06/12/2018            

In [72]:
# saved the final DF for each different year as a picklefile, which can then be accessed elsewhere and concatenated
with open("NewTrafficManhattanDF", 'wb') as picklefile: 
    pkl.dump(TrafficManhattanDF, picklefile)